Анализ тональности текста

In [0]:
import torch
import random
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
SEED = 0

In [0]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
!pip install transformers

     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 12.7MB/s 
     |████████████████████████████████| 3.8MB 16.7MB/s 
     |████████████████████████████████| 890kB 37.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=13e863b3d485c155188fde5e5830683173bf74718f9cbc65449c757b5ec9bd73
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
len(tokenizer.vocab)

30522

In [0]:
print(tokenizer.tokenize("I create my BERT model"))

['i', 'create', 'my', 'bert', 'model']


In [0]:
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I create my BERT model")))

[1045, 3443, 2026, 14324, 2944]


In [0]:
CLS = tokenizer.cls_token_id
SEP = tokenizer.sep_token_id
PAD = tokenizer.pad_token_id
UNK = tokenizer.unk_token_id

In [0]:
print(tokenizer.max_model_input_sizes)

{'bert-base-uncased': 512, 'bert-large-uncased': 512, 'bert-base-cased': 512, 'bert-large-cased': 512, 'bert-base-multilingual-uncased': 512, 'bert-base-multilingual-cased': 512, 'bert-base-chinese': 512, 'bert-base-german-cased': 512, 'bert-large-uncased-whole-word-masking': 512, 'bert-large-cased-whole-word-masking': 512, 'bert-large-uncased-whole-word-masking-finetuned-squad': 512, 'bert-large-cased-whole-word-masking-finetuned-squad': 512, 'bert-base-cased-finetuned-mrpc': 512, 'bert-base-german-dbmdz-cased': 512, 'bert-base-german-dbmdz-uncased': 512, 'bert-base-finnish-cased-v1': 512, 'bert-base-finnish-uncased-v1': 512, 'bert-base-dutch-cased': 512}


In [0]:
def sent2toks(sent):
  toks = tokenizer.tokenize(sent)
  toks = toks[:tokenizer.max_model_input_sizes['bert-base-uncased'] - 2]
  return toks

In [0]:
sent2toks('I create my BERT model')

['i', 'create', 'my', 'bert', 'model']

In [0]:
from torchtext import data

In [0]:
TEXT = data.Field(batch_first=True, use_vocab=False, tokenize=sent2toks, preprocessing=tokenizer.convert_tokens_to_ids, init_token=CLS, eos_token=SEP, pad_token=PAD, unk_token=UNK)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
from torchtext import datasets

In [0]:
data_train, data_test = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.4MB/s]


In [0]:
data_train, data_val = data_train.split(random_state=random.seed(SEED))

In [0]:
print(len(data_train), len(data_val), len(data_test))
print(vars(data_train.examples[0]))
print(tokenizer.convert_ids_to_tokens(vars(data_train.examples[0])['text']))

17500 7500 25000
{'text': [7939, 12638, 2003, 2055, 1996, 2069, 2518, 2008, 2003, 2157, 1999, 2023, 3185, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2672, 2320, 1999, 2019, 2220, 2754, 2023, 2001, 1037, 2488, 3185, 1012, 2619, 2787, 2000, 3013, 2070, 2895, 1998, 5436, 2685, 2046, 1996, 2927, 1997, 1996, 3185, 1010, 3228, 2185, 2087, 1997, 1996, 2466, 2240, 1999, 2055, 1996, 2034, 1019, 2781, 1012, 2008, 1998, 27853, 3649, 3857, 2039, 1999, 6393, 1998, 6348, 1996, 3185, 2453, 2031, 2018, 2077, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2061, 2034, 2009, 28679, 2015, 2017, 1998, 2059, 2009, 8509, 2017, 2125, 1012, 1996, 3689, 3775, 9276, 13956, 3458, 8636, 1997, 4487, 19022, 20806, 18697, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1997, 2607, 2045, 2003, 2008, 3110, 1997, 2307, 21321, 1998, 4963, 2008, 5691, 2066, 2023, 9280, 6133, 2000, 16021, 28345, 1999, 7193, 1012, 4379, 1010, 2009, 2515, 2008, 2061, 2065, 2017, 2024, 2559, 2005, 2008, 1012, 1012, 101

In [0]:
example = data_train.examples[0]
print(example)
print(vars(example))

{'text': [1000, 10911, 1997, 2293, 1000, 2003, 6947, 2008, 2025, 2296, 2272, 10265, 1011, 18427, 3166, 2040, 3594, 2892, 1011, 11225, 17330, 28592, 2066, 8101, 2003, 4276, 3773, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2030, 2672, 2242, 2001, 2439, 1999, 1996, 5449, 1997, 2023, 6789, 1997, 16266, 3567, 5602, 1010, 1037, 4012, 16969, 12418, 16185, 1011, 4427, 11170, 1997, 3183, 28101, 5794, 12782, 2758, 1024, 1000, 2010, 16371, 6651, 2094, 3110, 1998, 12266, 2773, 13068, 2150, 2124, 2004, 16266, 3567, 14066, 3351, 1012, 1000, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2096, 18062, 2061, 2099, 26531, 2038, 4569, 18737, 2093, 3816, 1011, 2039, 7472, 2015, 1010, 2014, 6471, 2535, 2347, 1005, 1056, 2130, 2039, 2000, 24188, 12083, 5740, 1999, 1000, 3510, 1997, 20965, 10464, 1012, 1000, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 2014, 4801, 1011, 12181, 2100, 9260, 2003, 15703, 1998, 2074, 3849, 2000, 5769, 2008, 1037, 2843, 1997, 3138, 2020, 2734, 2005, 2169, 214

In [0]:
LABEL.build_vocab(data_train)
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f796468c048>, {'pos': 0, 'neg': 1})


In [0]:
BATCH_SIZE = 128
train_it, val_it, test_it = data.BucketIterator.splits((data_train, data_val, data_test), batch_size=BATCH_SIZE, device=device)

In [0]:
from transformers import BertModel

In [0]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [0]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [0]:
import torch.nn as nn

In [0]:
class Analyzer(nn.Module):
  def __init__(self, bert, hid_dim, out_dim, num_lay, bi, drop):
    super().__init__()

    self.inp2emb = bert
    self.emb2hid = nn.GRU(bert.config.to_dict()['hidden_size'], hid_dim, num_layers=num_lay, bidirectional=bi, batch_first=True, dropout=drop)
    self.hid2out = nn.Linear(hid_dim * 2 if bi else hid_dim, out_dim)
    self.drop = nn.Dropout(drop)

  def forward(self, inp):
    with torch.no_grad():
      emb = self.inp2emb(inp)[0]
    
    hid, h = self.emb2hid(emb)
    
    if self.emb2hid.bidirectional:
      h = self.drop(torch.cat((h[-2,:,:], h[-1,:,:]), dim=1))
    else:
      h = self.drop(h[-1,:,:])
    
    out = self.hid2out(h)
    return out

In [0]:
HID_DIM = 256
OUT_DIM = 1
NUM_LAY = 2
BI = True
DROP = 0.25

analyzer = Analyzer(bert, HID_DIM, OUT_DIM, NUM_LAY, BI, DROP)

In [0]:
sum(p.numel() for p in analyzer.parameters() if p.requires_grad)

112241409

In [0]:
for name, param in analyzer.named_parameters():
  if name.startswith('inp2emb'):
    param.requires_grad = False

In [0]:
sum(p.numel() for p in analyzer.parameters() if p.requires_grad)

2759169

In [0]:
for name, param in analyzer.named_parameters():
  if param.requires_grad:
    print(name)

emb2hid.weight_ih_l0
emb2hid.weight_hh_l0
emb2hid.bias_ih_l0
emb2hid.bias_hh_l0
emb2hid.weight_ih_l0_reverse
emb2hid.weight_hh_l0_reverse
emb2hid.bias_ih_l0_reverse
emb2hid.bias_hh_l0_reverse
emb2hid.weight_ih_l1
emb2hid.weight_hh_l1
emb2hid.bias_ih_l1
emb2hid.bias_hh_l1
emb2hid.weight_ih_l1_reverse
emb2hid.weight_hh_l1_reverse
emb2hid.bias_ih_l1_reverse
emb2hid.bias_hh_l1_reverse
hid2out.weight
hid2out.bias


In [0]:
import torch.optim as optim

In [0]:
analyzer = analyzer.to(device)

In [0]:
opt = optim.Adam(analyzer.parameters())
crit = nn.BCEWithLogitsLoss().to(device)

In [0]:
def accuracy(out, trg):
  round_out = torch.round(torch.sigmoid(out))
  correct = (round_out == trg).float()
  return correct.sum() / len(correct)

In [0]:
def train(analyzer, it, opt, crit):
  epoch_loss = 0
  epoch_acc = 0

  analyzer.train()

  for batch in it:
    opt.zero_grad()
    out = analyzer(batch.text).squeeze(1)
    
    loss = crit(out, batch.label)
    acc = accuracy(out, batch.label)
    
    loss.backward()
    opt.step()
    
    epoch_loss += loss.item()
    epoch_acc += acc.item()
  
  return epoch_loss / len(it), epoch_acc / len(it)

In [0]:
def eval(analyzer, it, crit):
  epoch_loss = 0
  epoch_acc = 0

  analyzer.eval()

  with torch.no_grad():
    for batch in it:
      out = analyzer(batch.text).squeeze(1)
      
      loss = crit(out, batch.label)
      acc = accuracy(out, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()
  
  return epoch_loss / len(it), epoch_acc / len(it)

In [0]:
import time

In [0]:
def epoch_time(start, end):
  delta = end - start
  delta_min = int(delta / 60)
  delta_sec = int(delta - delta_min * 60)
  return delta_min, delta_sec

In [0]:
best = float('inf')

In [0]:
EPOCH_NUM = 5

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(analyzer, train_it, opt, crit)
  val_loss, val_acc = eval(analyzer, val_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if val_loss < best:
    best = val_loss
    torch.save(analyzer.state_dict(), '/content/gdrive/My Drive/Models/analyzer.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('VAL (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

Epoch: 0
Time: 6 m 41 s
Train (loss | acc): 0.2709144606207409 | 0.8904687997198453
VAL (loss | acc): 0.21823893323288127 | 0.9132889165716657
Epoch: 1
Time: 6 m 41 s
Train (loss | acc): 0.2328975436678768 | 0.9091409475263887
VAL (loss | acc): 0.25509013816461723 | 0.8996501453852249
Epoch: 2
Time: 6 m 40 s
Train (loss | acc): 0.21592530744136684 | 0.9131203389515842
VAL (loss | acc): 0.21046326423095443 | 0.919428802142709
Epoch: 3
Time: 6 m 40 s
Train (loss | acc): 0.18169378195583386 | 0.929585845801082
VAL (loss | acc): 0.21032538214477442 | 0.9190245876877995
Epoch: 4
Time: 6 m 41 s
Train (loss | acc): 0.1596207500972452 | 0.9390322518174665
VAL (loss | acc): 0.2178011897761943 | 0.9172195590148537


In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(analyzer, train_it, opt, crit)
  val_loss, val_acc = eval(analyzer, val_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if val_loss < best:
    best = val_loss
    torch.save(analyzer.state_dict(), '/content/gdrive/My Drive/Models/analyzer.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('VAL (loss | acc): ' + str(val_loss) + ' | ' + str(val_acc))

Epoch: 0
Time: 6 m 41 s
Train (loss | acc): 0.12688297161111867 | 0.9519398603996221
VAL (loss | acc): 0.24986769827240604 | 0.9139440231404062
Epoch: 1
Time: 6 m 41 s
Train (loss | acc): 0.10448787249896647 | 0.9619540424242506
VAL (loss | acc): 0.2553331854484849 | 0.916508697857291
Epoch: 2
Time: 6 m 40 s
Train (loss | acc): 0.08827883078560342 | 0.9685566090319279
VAL (loss | acc): 0.2997528127063129 | 0.9159372216564113
Epoch: 3
Time: 6 m 41 s
Train (loss | acc): 0.06645411002130186 | 0.9775790424242506
VAL (loss | acc): 0.2820703503438982 | 0.9158466215861045
Epoch: 4
Time: 6 m 41 s
Train (loss | acc): 0.060092365471170334 | 0.97854847673082
VAL (loss | acc): 0.30328134712526356 | 0.9167317131818351


In [0]:
analyzer.load_state_dict(torch.load('/content/gdrive/My Drive/Models/analyzer.pt'))

<All keys matched successfully>

In [0]:
test_loss, test_acc = eval(analyzer, test_it, crit)

In [0]:
print(test_loss, test_acc)

0.2018548963705496 0.9185985332849075


In [0]:
def analyze(analyzer, tokenizer, sent):
  analyzer.eval()
  toks = tokenizer.tokenize(sent)[:tokenizer.max_model_input_sizes['bert-base-uncased'] - 2]
  inds = [CLS] + tokenizer.convert_tokens_to_ids(toks) + [SEP]
  inp = torch.LongTensor(inds).unsqueeze(0).to(device)

  out = analyzer(inp).squeeze(1)
  return torch.sigmoid(out).item()

In [0]:
def analyze_list(analyzer, tokenizer, sents):
  outs = []
  for sent in sents:
    outs.append(analyze(analyzer, tokenizer, sent))
  return outs

In [0]:
analyze_list(analyzer, tokenizer, ['It is the best film I have ever seen', 'It was amazing!', 'It was OK', 'There are many films that are better', 'This film is strange', 'It was boring', 'It is the worst film I have ever seen'])

[0.002105788327753544,
 0.016265926882624626,
 0.13351258635520935,
 0.4152771234512329,
 0.8338457942008972,
 0.9776905179023743,
 0.9923290610313416]

In [167]:
analyze_list(analyzer, tokenizer, ["It's brilliant idea", "Sounds good", "I have doubts", "I'm confused", "I'm completely confused", "This show was absolutely awful"])

[0.01879182830452919,
 0.24711234867572784,
 0.40073511004447937,
 0.5724436044692993,
 0.8428231477737427,
 0.9892510175704956]

In [171]:
analyze(analyzer, tokenizer, "I'm completely happy")

0.008419858291745186

In [45]:
analyze(analyzer, tokenizer, "This movie is honestly the worst movie I've ever seen. Not only does it have terrible effects, you cant even hear the dialogue. Mystery Science Theater 3000 did this movie, and Joel Hodgson named this movie as the worst he's ever had to watch. He also said the movie was so hard to make fun of, that the cast couldn't even think of host segments to do based on the movie. And compared to the other things they have riffed, that's saying a lot. That's why this is the worst movie of all time.")

0.9949734210968018

In [0]:
label = torch.FloatTensor((1)).to(device)
label[0] = 0
analyze(analyzer, tokenizer, 'It was amazing', label)

(0.017056282609701157,
 tensor([-4.0540], device='cuda:0', grad_fn=<SqueezeBackward1>),
 tensor(0.0172, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 tensor(1., device='cuda:0'))

In [0]:
out, trg = torch.FloatTensor([[0], [1]]).to(device), torch.FloatTensor([[0], [1]]).to(device)
out[0][0] = 12
trg[0][0] = 1
crit(out, trg)

tensor(0.1706, device='cuda:0')

In [0]:
new_crit = nn.BCELoss().to(device)

In [0]:
out, trg = torch.FloatTensor([[0, 1]]).to(device), torch.FloatTensor([[0, 1]]).to(device)
new_crit(out, trg)

tensor(0., device='cuda:0')

In [0]:
out.shape

torch.Size([1, 2])